We are gonna revese it

In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb
import optuna
import torch

# Try to import TabPFN and TabM with fallbacks
try:
    from tabpfn import TabPFNRegressor
    TABPFN_AVAILABLE = True
except ImportError:
    print("⚠️  TabPFN not available. Install with: pip install tabpfn")
    TABPFN_AVAILABLE = False

# TabM proper import - check if it's available and get the correct class
try:
    import tabm
    # Try different possible imports for TabM
    if hasattr(tabm, 'TabMRegressor'):
        TabMRegressor = tabm.TabMRegressor
        TABM_AVAILABLE = True
    elif hasattr(tabm, 'TabM'):
        TabMRegressor = tabm.TabM
        TABM_AVAILABLE = True
    else:
        # Fallback: inspect the module to find available classes
        available_attrs = [attr for attr in dir(tabm) if not attr.startswith('_')]
        print(f"⚠️  TabM available attributes: {available_attrs}")
        TABM_AVAILABLE = False
except ImportError:
    print("⚠️  TabM not available. Install from GitHub: pip install git+https://github.com/yandex-research/tabm.git")
    TABM_AVAILABLE = False

print("Loading data...")
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train.drop([f'BlendProperty{i}' for i in range(1, 11)], axis=1)
y = train[[f'BlendProperty{i}' for i in range(1, 11)]]
X_test = test.drop(['ID'], axis=1)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

print("🚀 Starting Level 1 training...")

# Level 1: Advanced models as base models (with fallbacks)
level1_names = []
level1_oof = {}
level1_test = {}

# Add available models to level 1
available_models = []
if TABM_AVAILABLE:
    available_models.append('TabM')
if TABPFN_AVAILABLE:
    available_models.append('TabPFN')

# If neither advanced model is available, use strong traditional models
if not available_models:
    print("⚠️  Advanced models not available. Using traditional models for Level 1.")
    available_models = ['XGB', 'LGBM', 'RandomForest', 'ExtraTrees']

level1_names = available_models
level1_oof = {name: np.zeros(y.shape) for name in level1_names}
level1_test = {name: np.zeros((X_test.shape[0], y.shape[1])) for name in level1_names}

for t in range(y.shape[1]):
    print(f"🎯 Training Level 1 for target BlendProperty{t+1}...")
    for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
        print(f"  Fold {fold+1}/5")
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx, t], y.iloc[val_idx, t]

        # TabM (if available)
        if 'TabM' in level1_names and TABM_AVAILABLE:
            try:
                # Create TabM model - adjust parameters as needed
                tabm_model = TabMRegressor(
                    random_state=42,
                    device='cuda' if torch.cuda.is_available() else 'cpu'
                )
                tabm_model.fit(X_tr.values, y_tr.values)
                level1_oof['TabM'][val_idx, t] = tabm_model.predict(X_val.values)
                level1_test['TabM'][:, t] += tabm_model.predict(X_test.values) / kf.n_splits
            except Exception as e:
                print(f"    Error with TabM: {e}")
                # Remove TabM from level1_names if it fails
                if 'TabM' in level1_names:
                    level1_names.remove('TabM')

        # TabPFN (if available)
        if 'TabPFN' in level1_names and TABPFN_AVAILABLE:
            try:
                tabpfn = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
                tabpfn.fit(X_tr, y_tr.values)
                level1_oof['TabPFN'][val_idx, t] = tabpfn.predict(X_val)
                level1_test['TabPFN'][:, t] += tabpfn.predict(X_test) / kf.n_splits
            except Exception as e:
                print(f"    Error with TabPFN: {e}")
                if 'TabPFN' in level1_names:
                    level1_names.remove('TabPFN')

        # Fallback models if advanced models not available
        if 'XGB' in level1_names:
            xgb_model = xgb.XGBRegressor(random_state=42, verbosity=0)
            xgb_model.fit(X_tr, y_tr)
            level1_oof['XGB'][val_idx, t] = xgb_model.predict(X_val)
            level1_test['XGB'][:, t] += xgb_model.predict(X_test) / kf.n_splits

        if 'LGBM' in level1_names:
            lgb_model = lgb.LGBMRegressor(random_state=42, verbose=-1)
            lgb_model.fit(X_tr, y_tr)
            level1_oof['LGBM'][val_idx, t] = lgb_model.predict(X_val)
            level1_test['LGBM'][:, t] += lgb_model.predict(X_test) / kf.n_splits

        if 'RandomForest' in level1_names:
            rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
            rf.fit(X_tr, y_tr)
            level1_oof['RandomForest'][val_idx, t] = rf.predict(X_val)
            level1_test['RandomForest'][:, t] += rf.predict(X_test) / kf.n_splits

        if 'ExtraTrees' in level1_names:
            et = ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1)
            et.fit(X_tr, y_tr)
            level1_oof['ExtraTrees'][val_idx, t] = et.predict(X_val)
            level1_test['ExtraTrees'][:, t] += et.predict(X_test) / kf.n_splits

# Remove empty entries from level1 dictionaries
level1_names = [name for name in level1_names if name in level1_oof and level1_oof[name].sum() != 0]
level1_oof = {name: level1_oof[name] for name in level1_names}
level1_test = {name: level1_test[name] for name in level1_names}

print(f"\n📊 Level 1 MAPE Scores (using {len(level1_names)} models):")
for name in level1_names:
    mape = mean_absolute_percentage_error(y, level1_oof[name])
    print(f"  {name}: {mape:.6f}")

print("\n🔄 Preparing Level 2 inputs from Level 1 outputs...")
stack_X = np.concatenate([level1_oof[name] for name in level1_names], axis=1)
stack_X_test = np.concatenate([level1_test[name] for name in level1_names], axis=1)

print("\n🚀 Starting Level 2 stacking (Traditional ML Models)...")

# Level 2: Traditional ML models as stacking models
level2_names = [
    'Ridge', 'Lasso', 'ElasticNet', 'BayesianRidge', 'Huber',
    'RandomForest', 'ExtraTrees', 'AdaBoost', 'GradientBoost', 'Bagging',
    'KNN', 'SVR', 'XGB_L2', 'LGBM_L2'  # Added L2 suffix to avoid confusion
]
level2_oof = {name: np.zeros(y.shape) for name in level2_names}
level2_test = {name: np.zeros((X_test.shape[0], y.shape[1])) for name in level2_names}

for t in range(y.shape[1]):
    print(f"🎯 Level 2 stacking for BlendProperty{t+1}...")
    for fold, (tr_idx, val_idx) in enumerate(kf.split(stack_X)):
        print(f"  Fold {fold+1}/5")
        X_tr, X_val = stack_X[tr_idx], stack_X[val_idx]
        y_tr, y_val = y.iloc[tr_idx, t], y.iloc[val_idx, t]

        # Ridge
        ridge = Ridge(alpha=1.0)
        ridge.fit(X_tr, y_tr)
        level2_oof['Ridge'][val_idx, t] = ridge.predict(X_val)
        level2_test['Ridge'][:, t] += ridge.predict(stack_X_test) / kf.n_splits

        # Lasso
        lasso = Lasso(alpha=0.1, max_iter=1000)
        lasso.fit(X_tr, y_tr)
        level2_oof['Lasso'][val_idx, t] = lasso.predict(X_val)
        level2_test['Lasso'][:, t] += lasso.predict(stack_X_test) / kf.n_splits

        # ElasticNet
        elastic = ElasticNet(alpha=0.1, max_iter=1000)
        elastic.fit(X_tr, y_tr)
        level2_oof['ElasticNet'][val_idx, t] = elastic.predict(X_val)
        level2_test['ElasticNet'][:, t] += elastic.predict(stack_X_test) / kf.n_splits

        # BayesianRidge
        bayesian = BayesianRidge()
        bayesian.fit(X_tr, y_tr)
        level2_oof['BayesianRidge'][val_idx, t] = bayesian.predict(X_val)
        level2_test['BayesianRidge'][:, t] += bayesian.predict(stack_X_test) / kf.n_splits

        # Huber
        huber = HuberRegressor()
        huber.fit(X_tr, y_tr)
        level2_oof['Huber'][val_idx, t] = huber.predict(X_val)
        level2_test['Huber'][:, t] += huber.predict(stack_X_test) / kf.n_splits

        # RandomForest
        rf = RandomForestRegressor(n_estimators=50, random_state=42)
        rf.fit(X_tr, y_tr)
        level2_oof['RandomForest'][val_idx, t] = rf.predict(X_val)
        level2_test['RandomForest'][:, t] += rf.predict(stack_X_test) / kf.n_splits

        # ExtraTrees
        et = ExtraTreesRegressor(n_estimators=50, random_state=42)
        et.fit(X_tr, y_tr)
        level2_oof['ExtraTrees'][val_idx, t] = et.predict(X_val)
        level2_test['ExtraTrees'][:, t] += et.predict(stack_X_test) / kf.n_splits

        # AdaBoost
        ada = AdaBoostRegressor(random_state=42, n_estimators=50)
        ada.fit(X_tr, y_tr)
        level2_oof['AdaBoost'][val_idx, t] = ada.predict(X_val)
        level2_test['AdaBoost'][:, t] += ada.predict(stack_X_test) / kf.n_splits

        # GradientBoosting
        gb = GradientBoostingRegressor(random_state=42, n_estimators=100)
        gb.fit(X_tr, y_tr)
        level2_oof['GradientBoost'][val_idx, t] = gb.predict(X_val)
        level2_test['GradientBoost'][:, t] += gb.predict(stack_X_test) / kf.n_splits

        # Bagging
        bag = BaggingRegressor(random_state=42, n_estimators=50)
        bag.fit(X_tr, y_tr)
        level2_oof['Bagging'][val_idx, t] = bag.predict(X_val)
        level2_test['Bagging'][:, t] += bag.predict(stack_X_test) / kf.n_splits

        # KNN
        knn = KNeighborsRegressor(n_neighbors=5)
        knn.fit(X_tr, y_tr)
        level2_oof['KNN'][val_idx, t] = knn.predict(X_val)
        level2_test['KNN'][:, t] += knn.predict(stack_X_test) / kf.n_splits

        # SVR
        svr = SVR(kernel='rbf', gamma='scale')
        svr.fit(X_tr, y_tr)
        level2_oof['SVR'][val_idx, t] = svr.predict(X_val)
        level2_test['SVR'][:, t] += svr.predict(stack_X_test) / kf.n_splits

        # XGBoost (Level 2)
        xgb_model = xgb.XGBRegressor(random_state=42, verbosity=0)
        xgb_model.fit(X_tr, y_tr)
        level2_oof['XGB_L2'][val_idx, t] = xgb_model.predict(X_val)
        level2_test['XGB_L2'][:, t] += xgb_model.predict(stack_X_test) / kf.n_splits

        # LightGBM (Level 2)
        lgb_model = lgb.LGBMRegressor(random_state=42, verbose=-1)
        lgb_model.fit(X_tr, y_tr)
        level2_oof['LGBM_L2'][val_idx, t] = lgb_model.predict(X_val)
        level2_test['LGBM_L2'][:, t] += lgb_model.predict(stack_X_test) / kf.n_splits

print("\n📊 Level 2 MAPE Scores:")
level2_scores = {}
for name in level2_names:
    mape = mean_absolute_percentage_error(y, level2_oof[name])
    level2_scores[name] = mape
    print(f"  {name}: {mape:.6f}")

print("\n🔍 Starting Dynamic Level 3 optimization...")

# Dynamic selection of top performers (configurable top K models)
TOP_K = min(6, len(level2_names))  # Select top 6 models or all if less than 6
sorted_models = sorted(level2_scores.items(), key=lambda x: x[1])
top_models = [model[0] for model in sorted_models[:TOP_K]]

print(f"\n🎯 Selected top {len(top_models)} models for Level 3:")
for i, model in enumerate(top_models):
    print(f"  {i+1}. {model}: {level2_scores[model]:.6f}")

# Prepare Level 3 inputs with top performers
level3_oof = np.concatenate([level2_oof[model] for model in top_models], axis=1)
level3_test = np.concatenate([level2_test[model] for model in top_models], axis=1)

def objective(trial):
    # Dynamically suggest weights for each top model
    weights = {}
    for model in top_models:
        weights[model] = trial.suggest_float(f'w_{model}', 0.0, 1.0)
    
    # Normalize weights
    total_weight = sum(weights.values())
    if total_weight == 0:
        return float('inf')
    
    normalized_weights = {k: v/total_weight for k, v in weights.items()}
    
    # Create weighted ensemble
    ensemble_pred = sum(normalized_weights[model] * level2_oof[model] 
                       for model in top_models)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(y, ensemble_pred)
    return mape

# Optimize weights
print("🔧 Optimizing ensemble weights...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, show_progress_bar=True)

print(f"\n✅ Best MAPE: {study.best_value:.6f}")
print("🎯 Best weights:")
best_params = study.best_params
for param, value in best_params.items():
    print(f"  {param}: {value:.4f}")

# Normalize best weights
total_weight = sum(best_params.values())
normalized_weights = {k: v/total_weight for k, v in best_params.items()}

print("\n📊 Normalized weights:")
for param, value in normalized_weights.items():
    print(f"  {param}: {value:.4f}")

# Create final ensemble predictions
final_test = sum(normalized_weights[f'w_{model}'] * level2_test[model] 
                 for model in top_models)

# Final validation score
final_oof = sum(normalized_weights[f'w_{model}'] * level2_oof[model] 
                for model in top_models)

final_mape = mean_absolute_percentage_error(y, final_oof)
print(f"\n🎉 Final ensemble MAPE: {final_mape:.6f}")

print("\n📊 Individual target MAPE scores:")
for i in range(y.shape[1]):
    target_mape = mean_absolute_percentage_error(y.iloc[:, i], final_oof[:, i])
    print(f"  BlendProperty{i+1}: {target_mape:.6f}")

# Save submission with descriptive filename
submission = pd.DataFrame(final_test, columns=[f'BlendProperty{i}' for i in range(1, 11)])
submission.insert(0, 'ID', test['ID'])
submission_filename = f"submission_advanced_ensemble_top{TOP_K}.csv"
submission.to_csv(submission_filename, index=False)
print(f"\n💾 Submission file saved as '{submission_filename}'")

print(f"\n🎯 Advanced Dynamic Ensemble Summary:")
print(f"  Level 1: {len(level1_names)} advanced/strong models: {', '.join(level1_names)}")
print(f"  Level 2: {len(level2_names)} traditional ML models stacking on Level 1 outputs") 
print(f"  Level 3: Dynamic optimization with top {len(top_models)} performers: {', '.join(top_models)}")
print(f"  Final MAPE: {final_mape:.6f}")
print(f"  Submission saved as: {submission_filename}")

# Instructions for installing missing packages
print("\n📝 Installation Instructions:")
if not TABM_AVAILABLE:
    print("  For TabM: pip install git+https://github.com/yandex-research/tabm.git")
if not TABPFN_AVAILABLE:
    print("  For TabPFN: pip install tabpfn")

Loading data...
🚀 Starting Level 1 training...
🎯 Training Level 1 for target BlendProperty1...
  Fold 1/5
    Error with TabM: TabM.__init__() missing 1 required keyword-only argument: 'd_out'
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty2...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty3...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty4...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty5...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty6...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty7...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty8...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendPropert

[I 2025-07-23 03:35:01,984] A new study created in memory with name: no-name-f4a9d265-d420-4d5f-ba75-dbdf1fb0b45b



📊 Level 2 MAPE Scores:
  Ridge: 0.524448
  Lasso: 0.497542
  ElasticNet: 0.533723
  BayesianRidge: 0.524154
  Huber: 0.514973
  RandomForest: 0.560418
  ExtraTrees: 0.533252
  AdaBoost: 0.669838
  GradientBoost: 0.579397
  Bagging: 0.551338
  KNN: 2.019821
  SVR: 0.596871
  XGB_L2: 0.588618
  LGBM_L2: 0.647879

🔍 Starting Dynamic Level 3 optimization...

🎯 Selected top 6 models for Level 3:
  1. Lasso: 0.497542
  2. Huber: 0.514973
  3. BayesianRidge: 0.524154
  4. Ridge: 0.524448
  5. ExtraTrees: 0.533252
  6. ElasticNet: 0.533723
🔧 Optimizing ensemble weights...


Best trial: 10. Best value: 0.496592:   6%|▌         | 12/200 [00:00<00:01, 104.97it/s]

[I 2025-07-23 03:35:01,992] Trial 0 finished with value: 0.5018928440441229 and parameters: {'w_Lasso': 0.48372457343390274, 'w_Huber': 0.009338891243344527, 'w_BayesianRidge': 0.2429969951950708, 'w_Ridge': 0.6475425662382772, 'w_ExtraTrees': 0.9809345641198912, 'w_ElasticNet': 0.9346009907214455}. Best is trial 0 with value: 0.5018928440441229.
[I 2025-07-23 03:35:01,997] Trial 1 finished with value: 0.5031934975946918 and parameters: {'w_Lasso': 0.3423204508157036, 'w_Huber': 0.6930214108210238, 'w_BayesianRidge': 0.39578415644104104, 'w_Ridge': 0.08668982162536965, 'w_ExtraTrees': 0.7044508254506294, 'w_ElasticNet': 0.9952384018659869}. Best is trial 0 with value: 0.5018928440441229.
[I 2025-07-23 03:35:02,005] Trial 2 finished with value: 0.5142016152663127 and parameters: {'w_Lasso': 0.04903109849132603, 'w_Huber': 0.2660307190565857, 'w_BayesianRidge': 0.8893819530691486, 'w_Ridge': 0.8484223186256824, 'w_ExtraTrees': 0.00876521399375585, 'w_ElasticNet': 0.8774171923737691}. Bes

...

[I 2025-07-23 03:35:06,807] Trial 190 finished with value: 0.49134702353522447 and parameters: {'w_Lasso': 0.9971718867388674, 'w_Huber': 0.17899250553866788, 'w_BayesianRidge': 0.23269237795817566, 'w_Ridge': 0.019540246254081125, 'w_ExtraTrees': 0.4568221724022992, 'w_ElasticNet': 0.021608561011444358}. Best is trial 135 with value: 0.4908686601531828.
[I 2025-07-23 03:35:06,835] Trial 191 finished with value: 0.49138571784139734 and parameters: {'w_Lasso': 0.9810388784878883, 'w_Huber': 0.17969362281873513, 'w_BayesianRidge': 0.23482761056751555, 'w_Ridge': 0.019734721959229106, 'w_ExtraTrees': 0.45799209781199973, 'w_ElasticNet': 0.02240118037988768}. Best is trial 135 with value: 0.4908686601531828.
[I 2025-07-23 03:35:06,863] Trial 192 finished with value: 0.4914223613019824 and parameters: {'w_Lasso': 0.9965970981149906, 'w_Huber': 0.17183242839614038, 'w_BayesianRidge': 0.22898987460536208, 'w_Ridge': 0.03903847395481047, 'w_ExtraTrees': 0.4567258388181523, 'w_ElasticNet': 0.02

Best trial: 135. Best value: 0.490869:  98%|█████████▊| 197/200 [00:04<00:00, 34.61it/s]

[I 2025-07-23 03:35:06,891] Trial 193 finished with value: 0.4959810010374568 and parameters: {'w_Lasso': 0.9796439987257388, 'w_Huber': 0.1524854418887161, 'w_BayesianRidge': 0.2118072361671967, 'w_Ridge': 0.022802575189325772, 'w_ExtraTrees': 0.4541380364731208, 'w_ElasticNet': 0.4543295744244578}. Best is trial 135 with value: 0.4908686601531828.
[I 2025-07-23 03:35:06,918] Trial 194 finished with value: 0.4958220664822471 and parameters: {'w_Lasso': 0.9985635794748574, 'w_Huber': 0.12420018731103485, 'w_BayesianRidge': 0.8177431827205103, 'w_Ridge': 0.05166310736056151, 'w_ExtraTrees': 0.4779375528469234, 'w_ElasticNet': 0.022061656003442367}. Best is trial 135 with value: 0.4908686601531828.
[I 2025-07-23 03:35:06,947] Trial 195 finished with value: 0.49150083295718094 and parameters: {'w_Lasso': 0.9770043035265638, 'w_Huber': 0.1807873128411815, 'w_BayesianRidge': 0.24008089527741008, 'w_Ridge': 0.0008434208406057457, 'w_ExtraTrees': 0.5292693856518768, 'w_ElasticNet': 0.03816376

Best trial: 135. Best value: 0.490869: 100%|██████████| 200/200 [00:05<00:00, 39.45it/s]

[I 2025-07-23 03:35:06,999] Trial 197 finished with value: 0.4951647906241252 and parameters: {'w_Lasso': 0.9812520107802518, 'w_Huber': 0.14802649977286908, 'w_BayesianRidge': 0.28193920402352085, 'w_Ridge': 0.41879290812471437, 'w_ExtraTrees': 0.4618608712197463, 'w_ElasticNet': 0.08242313288382117}. Best is trial 135 with value: 0.4908686601531828.
[I 2025-07-23 03:35:07,026] Trial 198 finished with value: 0.49161347342324424 and parameters: {'w_Lasso': 0.9543322250909129, 'w_Huber': 0.09936968838165587, 'w_BayesianRidge': 0.1839331660269996, 'w_Ridge': 0.02202616688095308, 'w_ExtraTrees': 0.5521711106331125, 'w_ElasticNet': 0.05666701105357685}. Best is trial 135 with value: 0.4908686601531828.
[I 2025-07-23 03:35:07,056] Trial 199 finished with value: 0.49132094223224626 and parameters: {'w_Lasso': 0.9984540065437278, 'w_Huber': 0.22376026162913132, 'w_BayesianRidge': 0.22774106115834872, 'w_Ridge': 0.049201563802613486, 'w_ExtraTrees': 0.5149183713149603, 'w_ElasticNet': 0.000489